In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np

In [2]:
# first, create a TensorFlow constant
const = tf.constant(2.0, name="const")

# create TensorFlow variables
# b = tf.Variable(2.0, name='b')
b = tf.placeholder(tf.float32, [None, 1], name='b')
c = tf.Variable(1.0, name='c')

In [3]:
# now create some operations
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')

In [7]:
# setup the variable initialisation
init_op = tf.global_variables_initializer()

# start the session
with tf.Session() as sess:
    sess.run(init_op) # initialize the variables
    a_out = sess.run(a, feed_dict={b: np.arange(0, 10)[:, np.newaxis]})
    # a_out = sess.run(a) # compute the output of the graph
    print('Variable a is {}'.format(a_out))

Variable a is [[ 3.]
 [ 6.]
 [ 9.]
 [12.]
 [15.]
 [18.]
 [21.]
 [24.]
 [27.]
 [30.]]


In [8]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

root_logdir = "tf_basics"
logdir = "{}/run-{}/".format(root_logdir, now)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [13]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [120]:
train_images.shape

(60000, 28, 28)

In [131]:
train_images[0]

(28, 28)

In [123]:
train_labels.shape

(60000,)

In [127]:
train_images_flat = []
train_images_flat

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=uint8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=uint8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=uint8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=uint8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=uint8),
 array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0], dtype=uint8),
 array([  0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170,
        253, 253, 253, 253, 253, 225, 172, 253, 242, 195,  64,   0,   0,
          0,   0], dtype=uint8),
 array([  0,   0,   0,   0,   0,   0,   0,  49, 238, 253, 253, 253,

1680000

In [103]:
# Python optimiziation variables
learning_rate = 0.5
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [1])

In [125]:
# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 1], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([1]), name='b2')

In [112]:
# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

ValueError: Dimensions must be equal, but are 28 and 300 for 'Add_628' (op: 'Add') with input shapes: [300,28,28], [300].

In [113]:
# now calculate the hidden layer output - 
# in this case, a softmax activated output layer

y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

In [114]:
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))

In [115]:
# add an optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [116]:
# finally set up the initialization operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [117]:
# start the session

with tf.Session() as sess:
    # initialize the variables
    sess.run(init_op)
    total_batch = int(len(train_labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = train_images, train_labels
            np.expand_dims(batch_x, axis=0)
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
#     print(sess.run(accuracy, feed_dict={x: test_images, y: test_labels}))

InvalidArgumentError: Incompatible shapes: [60000] vs. [28]
	 [[node gradients_9/MatMul_18_grad/BroadcastGradientArgs (defined at /Users/austinkovach/Desktop/Odin/machine-learning-geron/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]

Original stack trace for 'gradients_9/MatMul_18_grad/BroadcastGradientArgs':
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/austinkovach/Desktop/Odin/machine-learning-geron/venv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/austinkovach/Desktop/Odin/machine-learning-geron/venv/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance